# Análisis con Specutils

![Specutils: Un paquete de Astropy para espectroscopía](data/specutils_logo.png)

Este notebook ofrece una visión general de algunas de las capacidades de análisis espectral del paquete coordinado de Astropy llamado Specutils. Aunque este notebook está diseñado como una introducción interactiva a Specutils en el momento de su escritura, la fuente de información más actualizada y completa sobre el paquete se encuentra en la documentación oficial:

https://specutils.readthedocs.io

Ten en cuenta que lo que se presenta a continuación asume que ya tienes conocimientos del material en el [notebook de introducción](Specutils_overview.ipynb). Si no es así, te recomendamos revisar ese notebook antes de continuar aquí.

## Importaciones

Comenzamos con algunas importaciones fundamentales para trabajar con `specutils` y realizar visualizaciones simples de espectros:

In [ ]:
import numpy as np

import astropy.units as u

import specutils
from specutils import Spectrum1D, SpectralRegion
specutils.__version__

In [ ]:
# para visualizar gráficos:
%matplotlib inline
import matplotlib.pyplot as plt

# para mostrar automáticamente las unidades físicas en los ejes:
from astropy.visualization import quantity_support
quantity_support();  # habilita el soporte para cantidades físicas en los gráficos

## Espectro de ejemplo y relación señal/ruido (SNR)

Para los ejemplos a continuación, también cargamos el espectro de muestra del SDSS que fue descargado en el [notebook de introducción](Specutils_overview.ipynb).

In [ ]:
from urllib.request import urlretrieve

# Definir la ruta local donde se guardará el archivo FITS del espectro SDSS
sdss_spectrum_path = 'data/sdss_spectrum.fits'

# URL del espectro SDSS a descargar
url = 'https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/1323/spec-1323-52797-0012.fits'

# Descargar el archivo desde la URL y guardarlo en la ruta local especificada
urlretrieve(url, sdss_spectrum_path)

# Leer el archivo FITS descargado en un objeto Spectrum1D con formato SDSS
sdss_spec = Spectrum1D.read(sdss_spectrum_path, format='SDSS-III/IV spec')

# Graficar el espectro usando un gráfico de escalones
plt.step(sdss_spec.wavelength, sdss_spec.flux);

Este archivo de ejemplo ya contiene incertidumbres, pero inicialmente están en forma de varianza inversa.
Convertimos esa información a desviación estándar para simplificar algunas de las operaciones más adelante, y también desactivamos el enmascaramiento en toda la región del espectro.

In [ ]:
sdss_spec.uncertainty.uncertainty_type

In [ ]:
from astropy.nddata import StdDevUncertainty

sdss_spec.uncertainty = StdDevUncertainty(sdss_spec.uncertainty.quantity**-0.5)
sdss_spec.uncertainty.uncertainty_type

In [ ]:
sdss_spec.mask[:] = False

Con estas incertidumbres, es sencillo calcular una de las medidas fundamentales de un espectro: la relación señal-a-ruido (SNR, por sus siglas en inglés) en todo el espectro.

In [ ]:
from specutils import analysis

analysis.snr(sdss_spec)

# Regiones Espectrales

La mayoría de los análisis espectrales requieren la especificación de una parte del espectro, por ejemplo, una línea espectral.  
Debido a que dichas regiones pueden tener valor por sí mismas independientemente de un espectro particular, se representan como objetos distintos del objeto del espectro en sí.
A continuación, se describen algunas formas en que dichas regiones pueden definirse.

In [ ]:
ha_region = SpectralRegion((6563-50)*u.AA, (6563+50)*u.AA)
ha_region

Las regiones también pueden definirse mediante valores de píxeles crudos (aunque, por supuesto, esto es más aplicable a un espectro específico):

In [ ]:
pixel_region = SpectralRegion(2100*u.pixel, 2600*u.pixel)
pixel_region

Además, *múltiples* regiones pueden estar en el mismo objeto `SpectralRegion`. Esto es útil, por ejemplo, para medir múltiples características espectrales en una sola llamada:

In [ ]:
HI_wings_region = SpectralRegion([(1.44*u.GHz, 1.43*u.GHz), (1.41*u.GHz, 1.4*u.GHz)])
HI_wings_region

Aunque las regiones son útiles para una variedad de pasos de análisis, fundamentalmente pueden usarse para extraer subespectros de espectros más grandes:

In [ ]:
from specutils.manipulation import extract_region

# Extraer una subregión del espectro original usando la región definida por píxeles
subspec = extract_region(sdss_spec, pixel_region)

# Graficar el espectro extraído usando un gráfico de escalones
plt.step(subspec.wavelength, subspec.flux)

# Calcular y mostrar la relación señal/ruido (SNR) del espectro extraído
analysis.snr(subspec)

# Medición de Líneas

Aunque el ajuste de líneas (detallado más adelante) es una buena opción para espectros con alta relación señal-ruido o cuando se desean cinemáticas detalladas, en la literatura a menudo se usan medidas más empíricas para espectros con más ruido o simplemente para procedimientos de análisis más simples. Specutils proporciona un conjunto de funciones para realizar este tipo de mediciones, así como estadísticas resumen similares sobre regiones espectrales. La [sección de análisis de la documentación de specutils](https://specutils.readthedocs.io/en/latest/analysis.html) ofrece una lista completa y ejemplos detallados, pero aquí demostramos algunos casos de ejemplo.

Nota: estas mediciones de líneas generalmente asumen que tu espectro está con el continuo restado o normalizado. Algunos pipelines espectrales hacen esto automáticamente, pero muchas veces no es así. Para nuestros ejemplos aquí, haremos este paso "a ojo", pero para una discusión más detallada sobre el modelado del continuo, consulta la siguiente sección. Basándonos en el gráfico anterior, estimamos un nivel de continuo para el área del espectro SDSS alrededor de la línea de emisión H-alfa, y usamos matemáticas básicas para construir los espectros normalizado y restado del continuo.

In [ ]:
# Estimar un nivel razonable de continuo para el área H-alfa del espectro
sdss_continuum = 205 * subspec.flux.unit

# Extraer la región de H-alfa del espectro y restar el nivel de continuo estimado
sdss_halpha_contsub = extract_region(sdss_spec, ha_region) - sdss_continuum

# Dibujar una línea horizontal en y=0 para referencia del continuo restado
plt.axhline(0, c='k', ls=':')

# Graficar el espectro con el continuo restado
plt.step(sdss_halpha_contsub.wavelength, sdss_halpha_contsub.flux)

# Establecer límites para el eje y para visualizar bien las fluctuaciones alrededor del continuo
plt.ylim(-50, 50)

Con el nivel de continuo identificado, ahora podemos hacer algunas mediciones de las líneas espectrales que se aprecian a simple vista — en particular, nos centraremos en la línea de emisión H-alfa. Aunque existen técnicas para identificar la línea automáticamente (ver la sección de ajuste más abajo), aquí asumimos que estamos haciendo procedimientos de "mirada rápida" donde la identificación manual es posible.

En la celda de abajo, cambia los valores de `LOWER` y `UPPER` para definir una región espectral que abarque justo la línea H-alfa (la del medio de las tres líneas). Puede ser útil cambiar los valores, volver a ejecutar la celda y ajustar de nuevo para "afinar" el número correcto.

In [ ]:
# Definir los límites inferior y superior para la región espectral que contiene las líneas de H-alfa
LOWER = 6000 * u.angstrom
UPPER = 7000 * u.angstrom
halpha_lines_region = SpectralRegion(LOWER, UPPER)

# Graficar el espectro con la región de interés resaltada
plt.step(sdss_halpha_contsub.wavelength, sdss_halpha_contsub.flux)

# Obtener los límites del eje y para mantener la escala después de sombrear
yl1, yl2 = plt.ylim()

# Rellenar la región seleccionada con una sombra ligera
plt.fill_between([halpha_lines_region.lower, halpha_lines_region.upper], 
                 yl1, yl2, alpha=.2)

# Volver a establecer los límites originales del eje y
plt.ylim(yl1, yl2)

Ahora puedes llamar a una variedad de funciones de análisis sobre el espectro con el continuo restado para estimar distintas propiedades de la línea (puedes ver la lista completa de funciones relevantes [en la sección de análisis de la documentación de specutils](https://specutils.readthedocs.io/en/stable/analysis.html#functions)).

In [ ]:
analysis.centroid(sdss_halpha_contsub, halpha_lines_region)

In [ ]:
analysis.fwhm(sdss_halpha_contsub, halpha_lines_region)

In [ ]:
analysis.line_flux(sdss_halpha_contsub, halpha_lines_region)

El ancho equivalente (EW), al ser una propiedad dependiente del continuo, puede calcularse directamente a partir del espectro si se conoce el nivel del continuo, o medirse sobre un espectro normalizado al continuo. Esto último es especialmente útil si el continuo no es uniforme en la región de la línea que se está midiendo.

In [ ]:
analysis.equivalent_width(sdss_spec, sdss_continuum, regions=halpha_lines_region)

In [ ]:
sdss_halpha_contnorm = sdss_spec / sdss_continuum
analysis.equivalent_width(sdss_halpha_contnorm, regions=halpha_lines_region)

## Ejercicio

Carga uno de los conjuntos de datos espectrales que creaste en los ejercicios del resumen en este cuaderno (es decir, tu propio conjunto de datos, uno descargado, o el blackbody con una característica espectral añadida artificialmente). Realiza una medición de flujo o de ancho de una línea en ese espectro directamente. ¿Notas algo extraño?

# Sustracción del Continuo

Aunque el ajuste del continuo para espectros a veces se considera más un "arte" que una ciencia, specutils proporciona herramientas para realizar diferentes enfoques de ajuste de continuo, sin hacer una recomendación específica sobre cuál es el "mejor" (ya que a menudo depende mucho de los datos). Más detalles están disponibles [en la sección relevante de la documentación de specutils](https://specutils.readthedocs.io/en/latest/fitting.html#continuum-fitting), pero aquí describimos las dos opciones básicas que existen: una función "a menudo suficientemente buena" y una herramienta más personalizable que se apoya en los modelos de [`astropy.modeling`](http://docs.astropy.org/en/stable/modeling/index.html) para proporcionar su flexibilidad.

### La forma "a menudo suficientemente buena"

La función `fit_generic_continuum` proporciona una función que a menudo es suficiente para continuos razonablemente bien comportados, especialmente para aplicaciones de "vista rápida" o similares donde la alta precisión no es tan crítica. La función genera un modelo de continuo, que puede evaluarse en cualquier valor del eje espectral:

In [ ]:
from specutils.fitting import fit_generic_continuum

In [ ]:
# Ajustar el continuo usando el método genérico
generic_continuum = fit_generic_continuum(sdss_spec)

# Evaluar el modelo de continuo ajustado en los valores del eje espectral
generic_continuum_evaluated = generic_continuum(sdss_spec.spectral_axis)

# Graficar el espectro original junto con el continuo ajustado
plt.step(sdss_spec.spectral_axis, sdss_spec.flux)  # Espectro original
plt.plot(sdss_spec.spectral_axis, generic_continuum_evaluated)  # Continuo ajustado
plt.ylim(100, 300)  # Limitar el eje y para mejor visualización

(Note que en algunas versiones de astropy/specutils puede aparecer una advertencia que dice "El modelo es lineal en los parámetros" al ejecutar la celda anterior. Esto no es un problema a menos que el rendimiento sea una preocupación seria, en cuyo caso se requiere una mayor personalización.)

Con este modelo en mano, se pueden producir espectros con el continuo restado o normalizado usando manipulaciones espectrales básicas:

In [ ]:
# Crear espectros corregidos: uno con el continuo restado y otro con el continuo normalizado
sdss_gencont_sub = sdss_spec - generic_continuum(sdss_spec.spectral_axis)  # Espectro con el continuo restado
sdss_gencont_norm = sdss_spec / generic_continuum(sdss_spec.spectral_axis)  # Espectro con el continuo normalizado

# Crear un gráfico con dos subgráficos (uno encima del otro)
ax1, ax2 = plt.subplots(2, 1)[1]

# Graficar el espectro con el continuo restado
ax1.step(sdss_gencont_sub.wavelength, sdss_gencont_sub.flux)
ax1.set_ylim(-50, 50)
ax1.axhline(0, color='k', ls=':')  # El continuo debe estar en flujo = 0

# Graficar el espectro con el continuo normalizado
ax2.step(sdss_gencont_norm.wavelength, sdss_gencont_norm.flux)
ax2.set_ylim(0, 2)
ax2.axhline(1, color='k', ls='--')  # El continuo debe estar en flujo = 1

### La forma personalizable

La función `fit_continuum` funciona de manera similar a `fit_generic_continuum`, pero está diseñada para que puedas proporcionar tu modelo de continuo favorito en lugar de estar ajustada a un modelo específico. Para ver la lista de modelos, consulta la [documentación de astropy.modeling](http://docs.astropy.org/en/stable/modeling/index.html).

In [ ]:
from specutils.fitting import fit_continuum
from astropy.modeling import models

Por ejemplo, supongamos que quieres usar un polinomio de Chebyshev de grado 3 como tu modelo de continuo. Puedes usar `fit_continuum` para obtener un objeto que se comporte igual que con `fit_generic_continuum`:

In [ ]:
# Ajuste del continuo usando un polinomio de Chebyshev de grado 3
chebdeg3_continuum = fit_continuum(sdss_spec, models.Chebyshev1D(3))

# Evaluar el modelo genérico de continuo previamente ajustado (si ya fue definido)
generic_continuum_evaluated = generic_continuum(sdss_spec.spectral_axis)

# Graficar el espectro original
plt.step(sdss_spec.spectral_axis, sdss_spec.flux)

# Graficar el continuo ajustado con el polinomio de Chebyshev de grado 3
plt.plot(sdss_spec.spectral_axis, chebdeg3_continuum(sdss_spec.spectral_axis))

# Limitar el eje Y para facilitar la visualización
plt.ylim(100, 300);

Esto proporciona total flexibilidad. Por ejemplo, también puedes probar otros polinomios como los polinomios de Hermite de grado más alto:

In [ ]:
# Ajustar el continuo usando polinomios de Hermite de grado 7 y 17
hermdeg7_continuum = fit_continuum(sdss_spec, models.Hermite1D(degree=7))
hermdeg17_continuum = fit_continuum(sdss_spec, models.Hermite1D(degree=17))

# Graficar el espectro original
plt.step(sdss_spec.spectral_axis, sdss_spec.flux)

# Graficar los diferentes modelos de continuo ajustados
plt.plot(sdss_spec.spectral_axis, chebdeg3_continuum(sdss_spec.spectral_axis))       # Polinomio de Chebyshev de grado 3
plt.plot(sdss_spec.spectral_axis, hermdeg7_continuum(sdss_spec.spectral_axis))       # Polinomio de Hermite de grado 7
plt.plot(sdss_spec.spectral_axis, hermdeg17_continuum(sdss_spec.spectral_axis))      # Polinomio de Hermite de grado 17

# Limitar el eje Y para una mejor visualización
plt.ylim(150, 250);

Esto demuestra de inmediato las compensaciones en el ajuste polinómico: mientras que los polinomios de alto grado capturan mejor las oscilaciones del espectro que los de bajo grado, también *sobreajustan* cerca de las líneas de emisión fuertes.

## Ejercicio

Intenta combinar la funcionalidad de `SpectralRegion` y el ajuste de continuo para ajustar únicamente las partes del espectro que *son* continuo (es decir, sin incluir las líneas de emisión). ¿Puedes mejorar el resultado?

## Ejercicio

Usando el espectro del ejercicio anterior, primero sustrae un continuo y luego vuelve a hacer la medición. ¿Es mejor el resultado?

# Ajuste de Líneas

Además de las mediciones más empíricas descritas arriba, `specutils` ofrece herramientas para realizar ajustes de líneas espectrales. El enfoque es similar al del modelado del continuo: se ajustan modelos de [astropy.modeling](http://docs.astropy.org/en/stable/modeling/index.html) al espectro, y esos modelos pueden usarse directamente, o bien sus parámetros.

In [ ]:
from specutils import fitting

El mecanismo de ajuste primero debe recibir estimaciones para las ubicaciones de las líneas. Este proceso puede automatizarse usando funciones diseñadas para identificar líneas (más detalles sobre las opciones están [en la documentación](https://specutils.readthedocs.io/en/latest/fitting.html#line-finding)). Para conjuntos de datos donde estos algoritmos no son ideales, puedes sustituirlos por tus propias estimaciones (es decir, omitir este paso y comenzar con las ubicaciones de las líneas ya estimadas).

Aquí identificamos las tres líneas cerca de la región H-alpha en nuestro espectro SDSS, encontrando las líneas por encima de un umbral de flujo de aproximadamente $\sim 3 \sigma$. Luego se muestran como una tabla de astropy:

In [ ]:
# Identificar líneas espectrales en el espectro H-alfa con un umbral de detección de ~3 sigma
halpha_lines = fitting.find_lines_threshold(sdss_halpha_contsub, 3)

# Graficar el espectro H-alfa continuo-substraído
plt.step(sdss_halpha_contsub.spectral_axis, sdss_halpha_contsub.flux, where='mid')

# Dibujar líneas verticales en las posiciones centrales de cada línea detectada
for line in halpha_lines:
    plt.axvline(line['line_center'], color='k', ls=':')

# Mostrar la tabla de líneas detectadas
halpha_lines

(Si ves una advertencia sobre la relación señal-ruido, puedes ignorarla o seguir las instrucciones que indica para suprimir la advertencia. Esto ocurre porque nuestro recorte tiene mucho flujo real, por lo que *podría* ser que olvidamos restar el continuo.)

Ahora, para cada una de estas líneas, necesitamos ajustar un modelo. A veces es suficiente crear un modelo donde el centro esté en la línea y extraer el área apropiada de la línea para hacer una estimación de la misma. Esto no es *tan* sensible al tamaño de la región, al menos para líneas bien separadas como estas. El resultado es una lista de modelos que incluyen los detalles del ajuste:

In [ ]:
halpha_line_models = []
for line in halpha_lines:
    # Definir una región espectral centrada en la línea, con un ancho de ±5 Ångströms
    line_region = SpectralRegion(line['line_center']-5*u.angstrom,
                                 line['line_center']+5*u.angstrom)
    
    # Extraer esa región del espectro continuo-substraído
    line_spectrum = extract_region(sdss_halpha_contsub, line_region)
    
    # Crear un modelo Gaussiano con el centro inicial en la posición de la línea
    line_estimate = models.Gaussian1D(mean=line['line_center'])
    
    # Ajustar el modelo a los datos espectrales de la línea
    line_model = fitting.fit_lines(line_spectrum, line_estimate)
    
    # Agregar el modelo ajustado a la lista
    halpha_line_models.append(line_model)

# Graficar el espectro observado
plt.step(sdss_halpha_contsub.spectral_axis, sdss_halpha_contsub.flux, where='mid')

# Evaluar y graficar cada modelo ajustado sobre el espectro original
for line_model in halpha_line_models:
    evaluated_model = line_model(sdss_halpha_contsub.spectral_axis)
    plt.plot(sdss_halpha_contsub.spectral_axis, evaluated_model)  

# Mostrar la lista de modelos ajustados
halpha_line_models

Para modelos o ajustes más complejos, puede ser mejor usar la función `estimate_line_parameters` en lugar de crear manualmente, por ejemplo, un modelo `Gaussian1D` y establecer el centro. A continuación se muestra un ejemplo de este patrón.

Nota: En el ejemplo anterior proporcionamos un modelo `Gaussian1D` por defecto a la función `estimate_line_parameters`. Esta función hace estimaciones razonables de parámetros para modelos `Gaussian1D`, `Voigt1D` y `Lorentz1D`, que son los perfiles de línea más comunes utilizados para líneas espectrales, pero puede que no funcione igual de bien con otros modelos. Consulta [la sección relevante de la documentación](https://specutils.readthedocs.io/en/latest/fitting.html#parameter-estimation) para más detalles.

En este ejemplo también se muestra cómo realizar un ajuste *conjunto* de las tres líneas al mismo tiempo. Aunque la diferencia puede parecer sutil, en casos de líneas espectrales mezcladas (blended lines), este enfoque típicamente proporciona resultados mucho mejores:

In [ ]:
halpha_line_estimates = []
for line in halpha_lines:
    # Definir una región espectral centrada en la línea, con un ancho de ±3 Ångströms
    line_region = SpectralRegion(line['line_center']-3*u.angstrom,
                                 line['line_center']+3*u.angstrom)
    
    # Extraer esa región del espectro
    line_spectrum = extract_region(sdss_halpha_contsub, line_region)
    
    # Estimar los parámetros iniciales del modelo Gaussian1D para esa línea
    line_estimate = fitting.estimate_line_parameters(line_spectrum, models.Gaussian1D())
    
    halpha_line_estimates.append(line_estimate)

# Esto podría hacerse de manera más flexible con un bucle for, pero aquí lo hacemos explícitamente por simplicidad
combined_model_estimate = halpha_line_estimates[0] + halpha_line_estimates[1] + halpha_line_estimates[2]
combined_model_estimate

In [ ]:
# Ajustar un modelo combinado a la región H-alfa con la estimación inicial de modelos
combined_model = fitting.fit_lines(sdss_halpha_contsub, combined_model_estimate)

# Graficar el espectro H-alfa continuum-substraído usando un gráfico de escalones
plt.step(sdss_halpha_contsub.spectral_axis, sdss_halpha_contsub.flux, where='mid')

# Graficar el modelo combinado ajustado sobre el mismo eje espectral
plt.plot(sdss_halpha_contsub.spectral_axis, 
         combined_model(sdss_halpha_contsub.spectral_axis))  
    
# Mostrar el modelo combinado ajustado
combined_model

## Ejercicio

Ajusta una característica espectral de tu propio espectro usando los métodos de ajuste descritos arriba. Prueba los diferentes tipos de perfiles de línea (Gaussiano, Lorentziano o Voigt). Si estás usando el espectro de cuerpo negro (donde conoces la respuesta "verdadera" para la línea espectral), compara tu resultado con la respuesta verdadera.